In [4]:
from dotenv import load_dotenv
import tiktoken
import openai
import pandas as pd
import os
import numpy as np
from openai.embeddings_utils import cosine_similarity, get_embedding

In [5]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

#### Read CSV using pandas

In [6]:
input_datapath = "./data/recipes_onecol.csv"  
df = pd.read_csv(input_datapath)

In [7]:
df.head()

,Recipe
0,Recipe: Caramelized Bread Pudding with Chocola...
1,Recipe: White Sauce or Bechamel Sauce - Ingred...
2,"Recipe: Grilled Whole Mackerel with Lemon, Ore..."
3,Recipe: Christmas Croquembouche - Ingredients:...
4,Recipe: Chocolate Roll-Out Cookies - Ingredien...


### Using OpenAI model ada for embeddings

OpenAI offers one second-generation embedding model (denoted by -002 in the model ID) and 16 first-generation models (denoted by -001 in the model ID).
We recommend using text-embedding-ada-002 for nearly all use cases. It’s better, cheaper, and simpler to use. 

#### Initialize Embeddings model 

In [25]:
# embedding model parameters
# encoding for text-embedding-ada-002
embedding_encoding = "cl100k_base"  
# the maximum for text-embedding-ada-002 is 8191
max_tokens = 8000 
# the number of reviews to embed
#in case of rate limit error, reduce the number of reviews
top_n = 150
encoding = tiktoken.get_encoding(embedding_encoding)

#### Count number of tokens in each row

In [26]:
# skip Recipes that are too long to embed > max_tokens
df["n_tokens"] = df.Recipe.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)

In [27]:
df.head()

,Recipe,n_tokens
349,Recipe: Spaghetti Limone Parmeggiano - Ingredi...,463
350,Recipe: Teeny-Weeny Coxinha - Ingredients: 1 q...,832
351,Recipe: Almond-Crusted Shrimp Cakes with Lemon...,817
352,Recipe: Rosemary Olive Oil Bread - Ingredients...,1688
353,Recipe: Blackberry Buttermilk Panna Cottas wit...,692


#### Create embeddings

In [28]:
# This may take a few minutes
df["embedding"] = df.Recipe.apply(lambda x: get_embedding(x, engine=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME))
# save the file
df.to_csv("./data/recipes_onecol_with_embeddings.csv")

RetryError: RetryError[<Future at 0x2901c3460 state=finished raised RateLimitError>]

#### Read embeddings

In [8]:
datafile_path = "./data/recipes_onecol_with_embeddings.csv"
df = pd.read_csv(datafile_path)

# convert the string representation of the embedding to a numpy array
df["embedding"] = df.embedding.apply(eval).apply(np.array)

#### Define a function to ask a question and get an answer from GPT-3.5

In [9]:
def ask_question(question, n=1):
    # get embedding for question
    question_embedding = get_embedding(
        question,
        engine=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME
    )

    # find the most similar embedding in the dataset
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, question_embedding))

    # sort the results by similarity
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )
    
    # if we asked for more than one anwser (n=1 or more), we would return a list of answers
    return (results.similarity.iloc[0], ' '.join(results.Recipe.tolist()) )
    

#### Berries is similar to raspberries

In [10]:
answer = ask_question("Chocolate and berries")
print(f"Similarity Score: {answer[0]}, {answer[1]}")

Similarity Score: 0.8284768751105671, Recipe: Raspberry Sundaes with Chocolate Sauce and Roasted Almonds - Ingredients: 6 ounces bittersweet chocolate, finely chopped;3/4 cup water;2/3 cups sugar;2 tablespoons honey;3 tablespoons unsalted butter;Pinch of kosher salt;1 pint fresh raspberries (12 ounces);2 tablespoons sugar;2 tablespoons elderflower liqueur, such as St-Germain;2 tablespoons fresh lemon juice;2 pints vanilla ice cream;Chopped roasted unsalted almonds, for garnish- Instructions: To make the chocolate sauce, place a saucepan over medium-low heat, combine the chocolate, and water, vigorously stirring until completely melted and smooth. Let it come to a low simmer until small bubbles appear around the outside. Add sugar, honey and keep at a low simmer until thickened, about 3 minutes. Remove from heat and stir in butter and salt. Chocolate sauce will smooth out at is cools to room temperature. Store in an airtight container. In a bowl, gently toss together the raspberries, su

#### Typo: Bukgur instead of Bulgur

In [11]:
answer = ask_question("Bukgur vegan recipe")
print(f"Similarity Score: {answer[0]}, {answer[1]}")

Similarity Score: 0.8508641994464585, Recipe: Bulgur Veggie Burgers with Lime Mayonnaise - Ingredients: 1/2 cup chopped onion, divided;1 tablespoon olive oil plus additional for brushing;1/2 cup bulgur;1 cup water;1 cup canned pinto beans, rinsed and drained;1 1/2 tablespoon soy sauce;3/4 cup walnuts (2 1/2 ounces);2 garlic cloves, coarsely chopped;1/2 cup packed cilantro sprigs;3/4 teaspoon ground cumin;1/4 teaspoon cayenne;1/4 cup mayonnaise;1/4 teaspoon grated lime zest;1/2 teaspoon fresh lime juice;4 slices multi-grain bread,toasted;Equipment: a perforated grill sheet;Accompaniments: lettuce; sliced tomato- Instructions: Cook half of onion with 1/4 teaspoon salt in oil in a small heavy saucepan over medium heat, stirring occasionally, until golden, 5 to 7 minutes. Add bulgur and water and cook, covered, over low heat until water is absorbed, 15 to 18 minutes. Transfer to a bowl and stir in beans and soy sauce. Pulse bulgur mixture, walnuts, garlic, cilantro, cumin, cayenne, a round

In [34]:
answer = ask_question("Fish no alcohol")
print(f"Similarity Score: {answer[0]}, {answer[1]}")

Similarity Score: 0.7880176978357717, Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 

In [35]:
answer = ask_question("Fish with wine")
print(f"Similarity Score: {answer[0]}, {answer[1]}")

Similarity Score: 0.8261893886663239, Recipe: Trout in Riesling - Ingredients: 2 tablespoons (1 oz) unsalted butter plus additional for greasing;1 medium carrot;1 celery rib;4 (6- to 8-oz) trout fillets with skin;1 1/2 teaspoons salt;1 teaspoon black pepper;4 medium shallots, cut lengthwise into 1/8-inch-thick slices (1 cup);2 fresh parsley sprigs;5 black peppercorns;1 1/2 cups water;1 1/2 cups dry Riesling;3/4 cup heavy cream;2 teaspoons cornstarch;2 teaspoons chopped fresh tarragon;1/2 teaspoon fresh lemon juice;Special equipment: an adjustable-blade slicer; tweezers or needlenose pliers- Instructions: Put oven rack in middle position and preheat oven to 375°F. Butter bottom and sides of a 13- by 9-inch glass baking dish. Cut carrot and celery lengthwise with slicer into very thin matchsticks (less than 1/8 inch wide and 2 1/2 inches long). Rinse trout and remove any pin bones with tweezers or needlenose pliers. Pat dry, then sprinkle 1/2 teaspoon each of salt and pepper on fish (fle

In [36]:

answer = ask_question("Fish with alcohol")
print(f"Similarity Score: {answer[0]}, {answer[1]}")

Similarity Score: 0.8070105198549736, Recipe: Soft Fish Tacos - Ingredients: 1/4 cup mayonnaise;1/4 cup ketchup;1/4 cup crema mexicana*;1 cup all purpose flour;1 teaspoon fine sea salt;1/2 teaspoon ground pepper;1 cup dark beer, room temperature;13/4 pounds halibut, cut into 5x3/4-inch strips;1 lime, halved crosswise;12 white corn tortillas;Vegetable oil (for deep-frying);1 1/2 cups shredded red cabbage;2 large tomatoes, chopped;Lime wedges;Bottled hot pepper sauce- Instructions: For sauce: Mix all ingredients in bowl; season with salt and pepper.
Mix all ingredients in bowl; season with salt and pepper.
For batter and fish: Whisk flour, salt, and pepper in bowl; pour in beer, whisking until batter is smooth. Let stand 15 minutes. Sprinkle fish with salt and pepper. Squeeze some lime juice over each strip. Let stand 15 minutes. Mix fish into batter. Preheat oven to 200°F. Heat skillet over medium heat. Stack 2 tortillas. Sprinkle top with water. Place in skillet, wet side down. Heat 1 

In [37]:

answer = ask_question("Meat with alcohol")
print(f"Similarity Score: {answer[0]}, {answer[1]}")

Similarity Score: 0.8139065682641311, Recipe: Beer-Braised Holiday Top of the Rib - Ingredients: 1 tablespoon sweet or hot smoked paprika;1 tablespoon dark brown sugar;1 tablespoon ground cumin;1 tablespoon instant coffee granules;1 teaspoon kosher salt;1 teaspoon freshly ground black pepper;1/2 teaspoon onion powder;1/2 teaspoon garlic powder;One 5-pound top of the rib;3 tablespoons olive oil;Three 12-ounce cans or bottles dark lager beer;1 garlic head, halved horizontally;1 pound small parsnips, peeled and halved length wise;1 pound peeled baby carrots with greens- Instructions: 1. Preheat the oven to 325°F 2. Combine the paprika, brown sugar, cumin, coffee, salt, pepper, onion powder, and garlic powder in a small bowl and stir to mix. Pat the meat dry with paper towels and rub the spice mixture all over. 3. Heat the olive oil in a large Dutch oven over medium-high heat. Sear the meat until nicely browned, 4 to 6 minutes per side. Pour the beer over the meat, cover, and transfer to t